# Deploying Machine Learning Models on GCP Kubernetes (with RBAC)

<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequisites
 - You need a running GCP cluster with kubernetes>1.8 with kubectl configured to use.
 -  If you wish to test the JSON schema checks you will need presently to enbale "alpha features" for your cluster (Jan 2018).
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [Helm](https://github.com/kubernetes/helm)
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)

## Install helm

In [1]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount "tiller" created
clusterrolebinding "tiller" created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [2]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

deployment "tiller-deploy" successfully rolled out


## Start Seldon-Core

In [3]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd \
    --set usage_metrics.enabled=true

NAME:   seldon-core-crd
LAST DEPLOYED: Wed Sep 12 17:53:13 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  1s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     1s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  1s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          1s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        1s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  1s


NOTES:
NOTES: TODO




In [6]:
!kubectl create namespace seldon

namespace "seldon" created


In [7]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "gke_kubeflow-testing-213813_europe-west1-b_cluster-1" modified.


In [8]:
!helm install ../helm-charts/seldon-core --name seldon-core \
        --namespace seldon

NAME:   seldon-core
LAST DEPLOYED: Wed Sep 12 17:53:59 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/Deployment
NAME                                DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-core-seldon-apiserver        1        1        1           0          1s
seldon-core-seldon-cluster-manager  1        1        1           0          1s
seldon-core-redis                   1        1        1           0          1s

==> v1/Pod(related)
NAME                                                 READY  STATUS             RESTARTS  AGE
seldon-core-seldon-apiserver-68f5984b49-xqt6n        0/1    ContainerCreating  0         1s
seldon-core-seldon-cluster-manager-7bbc74c894-pkpkh  0/1    ContainerCreating  0         1s
seldon-core-redis-575979b79b-nl4rm                   0/1    ContainerCreating  0         1s

==> v1/ServiceAccount
NAME    SECRETS  AGE
seldon  1        1s

==> v1beta1/ClusterRole
NAME        AGE
seldon-crd  1s

==> v1/ClusterRoleBinding
NAME    AGE


In [9]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver

deployment "seldon-core-seldon-cluster-manager" successfully rolled out
deployment "seldon-core-seldon-apiserver" successfully rolled out


## Set up REST and gRPC methods

**Ensure you port forward the seldon api-server REST and GRPC ports**:

REST:
```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8002:8080
```

GRPC:
```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:5000
```

In [10]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

In [11]:
from seldon_utils import *
API_GATEWAY_REST="localhost:8002"
API_GATEWAY_GRPC="localhost:8003"

## Normal Operation

### Create Seldon Deployment

In [12]:
!kubectl create -f resources/model.json -n seldon

seldondeployment "seldon-model" created


Get the status of the SeldonDeployment. **When ready the replicasAvailable should be 1**.

In [13]:
!kubectl get seldondeployments seldon-model -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:test-deployment-example-svc-orch replicas:1 replicasAvailable:1] map[replicas:1 replicasAvailable:1 name:test-deployment-example-classifier-0]]]

### Get Predictions

#### REST Request

In [14]:
rest_request_api_gateway("oauth-key","oauth-secret",API_GATEWAY_REST)

{"access_token":"9d189357-7491-4fd5-918c-972dc07cff30","token_type":"bearer","expires_in":43199,"scope":"read write"}
{
  "meta": {
    "puid": "b4o703030ro6658n3s7tdcl87p",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.09212613536947716]
    }
  }
}


#### gRPC Request

In [15]:
grpc_request_api_gateway("oauth-key","oauth-secret",API_GATEWAY_REST,API_GATEWAY_GRPC)

{"access_token":"9d189357-7491-4fd5-918c-972dc07cff30","token_type":"bearer","expires_in":43197,"scope":"read write"}
meta {
  puid: "4vmiol2sj12hst795cccd072g"
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08313732008580146
  }
}



## Update Deployment with Canary

In [13]:
!kubectl apply -f resources/model_with_canary.json -n seldon

seldondeployment "seldon-model" configured


Check the status of the deployments. Note: **Might need to run several times until replicasAvailable is 1 for both predictors**.

In [16]:
!kubectl get seldondeployments seldon-model -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:test-deployment-example-svc-orch replicas:1 replicasAvailable:1] map[replicas:1 replicasAvailable:1 name:test-deployment-example-classifier-0] map[replicasAvailable:1 name:test-deployment-canary-svc-orch replicas:1] map[name:test-deployment-canary-mean-classifier-0 replicas:1 replicasAvailable:1]]]

In [17]:
rest_request_api_gateway("oauth-key","oauth-secret",API_GATEWAY_REST)

{"access_token":"7f97c128-aa37-4b76-b2cc-4eea99237f34","token_type":"bearer","expires_in":43131,"scope":"read write"}
{
  "meta": {
    "puid": "aicmssrcgakld9jtf8e9jdgbr7",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [2, 1],
      "values": [0.07541984830478907, 0.08452793566856674]
    }
  }
}


In [18]:
grpc_request_api_gateway("oauth-key","oauth-secret",API_GATEWAY_REST,API_GATEWAY_GRPC)

{"access_token":"7f97c128-aa37-4b76-b2cc-4eea99237f34","token_type":"bearer","expires_in":43128,"scope":"read write"}
meta {
  puid: "88q5lvu9rbe3jq4kpfki2l6gcf"
}
data {
  names: "proba"
  tensor {
    shape: 2
    shape: 1
    values: 0.08780586679952734
    values: 0.0825496801645638
  }
}



## Tear Down

In [ ]:
!kubectl delete -f resources/model.json -n seldon

In [ ]:
!helm delete seldon-core --purge

In [ ]:
!helm delete seldon-core-crd --purge